# 재무 분석을 통한 주식거래전략
> 특정 종목에 대해 시장을 예측하는 주식

In [1]:
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
import os
load_dotenv()
# gpt = ChatOpenAI(model='gpt-3.5', api_key=os.getenv("OPENAI_API_KEY")) # Loading GPT-3.5
clud = ChatAnthropic(model="claude-3-5-sonnet-20240620",anthropic_api_key=os.getenv("Anthropic_API_KEY"), temperature=0)
groq = ChatGroq(model_name="llama3-70b-8192",groq_api_key="GROQ_API_KEY",temperature=0)
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## Agent 계획
- data_analyst_agent
- trading_strategy_agent
- risk_management_agent

In [30]:
from crewai_tools import tool
from crewai_tools import WebsiteSearchTool, SerperDevTool,ScrapeWebsiteTool,DirectoryReadTool,FileReadTool

search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrape_tool = ScrapeWebsiteTool()
docs_tool = DirectoryReadTool(directory='./financial-posts')
file_tool = FileReadTool()

class DataTools():
    @tool("get historical stock price")
    def get_stock_price(ticker: str, start_date: str, end_date: str) -> str:
        """
        Get the stock price for a given ticker symbol and date range.

        Args:
            ticker (str): {stock_selection} (e.g., 'AAPL' for Apple Inc.)
            start_date (str): The start date in the format 'YYYY-MM-DD'
            end_date (str): The end date in the format 'YYYY-MM-DD'

        Returns:
            str: The stock price data for the given ticker and date range.
        """        
        import yfinance as yf
        import datetime

        today = datetime.datetime.now()
        one_year_ago = today - datetime.timedelta(days=365)

        stock_data = yf.download(ticker, start=one_year_ago, end=today)
        stock_price_data = stock_data['Adj Close'].to_string()

        return stock_price_data

    @tool("get financial statement")        
    def get_financial_statement(ticker: str) -> str:
        '''
        Get the quarterly financial statement for a given ticker symbol.

        Args:
            ticker (str): {stock_selection} (e.g., 'AAPL' for Apple Inc.)

        Returns:
            str: quarterly_financials
        '''
        import pandas as pd
        import yfinance as yf
        ticker = yf.Ticker(ticker)
        return ticker.quarterly_financials

# def cache_func(args, result):
#     # Cache the result if the ticker is a valid stock symbol
#     ticker, start_date, end_date = args
#     try:
#         yf.download(ticker, start=start_date, end=end_date)
#         cache = True
#     except:
#         cache = False
#     return cache

# DataTools.get_stock_price.cache_function = cache_func
# DataTools.get_financial_statement.cache_function = cache_func


In [32]:
data_analyst_agent = Agent(
    role="data_analyst_agent",
    goal="실시간으로 시장 데이터를 모니터링하고 분석하여 "
         "트렌드를 식별하고 시장 움직임을 예측",
    backstory="금융 시장 전문가로서 "
              "통계 모델링과 기계 학습을 사용하여 "
              "중요한 통찰력을 제공합니다. 데이터 분석 능력으로"
              "거래 결정을 알리는 데"
              "핵심적인 역할을 합니다.",
    tools=[search_tool,DataTools.get_stock_price,DataTools.get_financial_statement],
    verbose=True,
    allow_delegation=True,
    llm=clud
)

In [33]:
trading_strategy_agent = Agent(
    role="trading_strategy_agent",
    goal="data_analyst_agent의 분석 기반, "
         "다양한 트레이딩 전략을 개발하고 테스트",
    backstory="금융 시장과 정량 분석에 대한 깊은 이해를 갖추고 "
              "트레이딩 전략을 고안하고 개선합니다. "
              "다양한 접근 방식의 성과를 평가하여 "
              "가장 수익성이 높고 위험 회피적인 옵션을 결정합니다.",
    tools=[scrape_tool],
    verbose=True,
    allow_delegation=True,
    llm=groq
)

In [34]:
risk_management_agent = Agent(
    role="risk_management_agent",
    goal="투자 전략에 있어서 잠재적 트레이딩 활동과 관련된 리스크를 "
         "평가하고 통찰력을 제공합니다.",
    backstory="리스크 평가 모델과 시장 동향에 대한 깊은 이해를 바탕으로 "
              "제안된 트레이드의 잠재적 리스크를 꼼꼼히 살펴봅니다. "
              "리스크 노출에 대한 상세한 분석을 제공하고 "
              "트레이딩 활동이 회사의 리스크 허용 범위 내에서 이루어지도록 "
              "보호 장치를 제안합니다.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    allow_delegation=True,
    llm=clud
)

## 작업 만들기

In [35]:
# 데이터 분석가 에이전트를 위한 작업: 시장 데이터 분석
data_analysis_task = Task(
    description=(
        "선택된 주식({stock_selection})에 대한 "
        "시장 데이터를 지속적으로 모니터링하고 분석."
        "통계 모델링과 기계 학습을 사용하여 "
        "트렌드를 식별하고 시장 움직임을 예측."
    ),
    expected_output=(
        "{stock_selection}에 대한 "
        "중요한 시장 기회 또는 위협에 관한 인사이트 및 경고"
    ),
    agent=data_analyst_agent,
)

In [36]:
# 트레이딩 전략 에이전트를 위한 작업: 트레이딩 전략 개발
strategy_development_task = Task(
    description=(
        "trading_strategy_agent의 통찰력과 "
        "사용자 정의 리스크 허용 범위({risk_tolerance})를 바탕으로 "
        "트레이딩 전략을 개발하고 개선하세요. "
        "트레이딩 선호도({trading_strategy_preference})를 고려하세요."
    ),
    expected_output=(
        "{stock_selection}에 대한 "
        "사용자의 리스크 허용 범위에 부합하는 "
        "잠재적 트레이딩 전략 세트입니다."
    ),
    agent=trading_strategy_agent,
)

In [37]:
# 리스크 어드바이저 에이전트를 위한 작업: 트레이딩 리스크 평가
risk_assessment_task = Task(
    description=(
        "{stock_selection}에 대한 제안된 트레이딩 전략과 "
        "실행 계획과 관련된 리스크를 평가하세요. "
        "잠재적 리스크에 대한 상세한 분석을 제공하고 "
        "리스크 완화 전략을 제안하세요."
    ),
    expected_output=(
        "{stock_selection}에 대한 잠재적 리스크와 "
        "리스크 완화 권고 사항을 상세히 설명한 "
        "포괄적인 리스크 분석 보고서입니다."
    ),
    agent=risk_management_agent,
)

## Crew 생성
- 'Process' 클래스는 워크플로를 상담원에게 위임하는 데 도움이 됩니다(직장에서의 매니저처럼).
- 아래 예시에서는 이를 계층적으로 실행합니다.
- 'manager_llm'을 사용하면 사용하려는 "manager" LLM을 선택할 수 있습니다.

In [38]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           risk_assessment_task],
    
    manager_llm=clud,

    process=Process.hierarchical,
    verbose=True
)

2024-06-23 10:59:03,248 - 21492 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Crew  실행

In [39]:
# 프로세스를 시작해봅시다
financial_trading_inputs = {
    'stock_selection': 'TSLA',
    'initial_capital': '2500',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Monthly Trading',
    'news_impact_consideration': True
}

In [40]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: 선택된 주식(TSLA)에 대한 시장 데이터를 지속적으로 모니터링하고 분석.통계 모델링과 기계 학습을 사용하여 트렌드를 식별하고 시장 움직임을 예측.


> Entering new CrewAgentExecutor chain...
Thought: To complete this task effectively, I need to delegate work to our data analyst and trading strategy agent. I'll start by asking the data analyst to gather and analyze market data for TSLA, and then have the trading strategy agent use that information to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "data_analyst_agent", "task": "Gather and analyze market data for TSLA", "context": "We need to continuously monitor and analyze market data for Tesla (TSLA) stock. Please collect relevant data such as stock price, trading volume, market cap, P/E ratio, and any other significant financial metrics. Also, gather news and sentiment data related to TSLA. Analyze this data to identify any notable patterns or anomalies."}


> Entering new Cr

[*********************100%%**********************]  1 of 1 completed


 

Date
2023-06-26    241.050003
2023-06-27    250.210007
2023-06-28    256.239990
2023-06-29    257.500000
2023-06-30    261.769989
2023-07-03    279.820007
2023-07-05    282.480011
2023-07-06    276.540009
2023-07-07    274.429993
2023-07-10    269.609985
2023-07-11    269.790009
2023-07-12    271.989990
2023-07-13    277.899994
2023-07-14    281.380005
2023-07-17    290.380005
2023-07-18    293.339996
2023-07-19    291.260010
2023-07-20    262.899994
2023-07-21    260.019989
2023-07-24    269.059998
2023-07-25    265.279999
2023-07-26    264.350006
2023-07-27    255.710007
2023-07-28    266.440002
2023-07-31    267.429993
2023-08-01    261.070007
2023-08-02    254.110001
2023-08-03    259.320007
2023-08-04    253.860001
2023-08-07    251.449997
2023-08-08    249.699997
2023-08-09    242.190002
2023-08-10    245.339996
2023-08-11    242.649994
2023-08-14    239.759995
2023-08-15    232.960007
2023-08-16    225.600006
2023-08-17    219.220001
2023-08-18    215.490005
2023-08-21    231

- 최종 결과를 마크다운으로 표시합니다.

In [41]:
from IPython.display import Markdown
Markdown(result)

Comprehensive Risk Analysis Report for TSLA Trading Strategies

1. Volatility Risk:
TSLA stock has demonstrated significant volatility, with a 30% decline in 2024 following a 102% gain in 2023. This volatility poses risks for all proposed strategies, particularly the Trend-Following Monthly Strategy and Monthly Momentum-Based Strategy.

Mitigation Strategies:
a) Implement strict stop-loss orders (15-20% trailing) as suggested in the original implementation guidelines.
b) Utilize options strategies such as protective puts or collars to limit downside risk.
c) Adjust position sizing based on volatility, reducing exposure during high volatility periods.
d) For the Trend-Following Monthly Strategy, consider using multiple moving averages (e.g., 20-day, 50-day, and 200-day) to confirm trends and reduce false signals.
e) In the Monthly Momentum-Based Strategy, incorporate volatility indicators (e.g., Average True Range) to adjust position sizes dynamically.

2. Valuation Risk:
Tesla's high valuation and questioned growth story may lead to sudden price corrections, affecting strategies like Monthly Rebalancing and Dollar-Cost Averaging.

Mitigation Strategies:
a) Regularly reassess Tesla's fundamentals and adjust position sizes accordingly.
b) Implement a gradual entry strategy for Dollar-Cost Averaging to spread out the risk of buying at peak valuations.
c) Consider pairing Tesla with value stocks in the Sector Rotation Strategy to balance growth and value exposure.
d) For the Monthly Rebalancing Strategy, set a maximum allocation percentage (e.g., 7% of the portfolio) to prevent overexposure during periods of rapid price increase.

3. Market Sentiment Risk:
News-driven volatility can significantly impact the News-Driven Monthly Adjustment Strategy and Earnings-Focused Quarterly Strategy.

Mitigation Strategies:
a) Develop a robust news analysis framework to filter out noise from significant news events.
b) Implement a time delay between news events and trading decisions to avoid knee-jerk reactions.
c) Use sentiment analysis tools to gauge overall market sentiment before making adjustments.
d) For the News-Driven Monthly Adjustment Strategy, establish clear criteria for what constitutes "significant" news and how it should impact position sizing.
e) In the Earnings-Focused Quarterly Strategy, consider using options strategies (e.g., straddles or strangles) around earnings announcements to capitalize on volatility while limiting risk.

4. Sector-Specific Risks:
The electric vehicle (EV) market faces challenges such as competition, battery supply issues, and potential changes in government subsidies, which can affect Tesla's performance.

Mitigation Strategies:
a) Diversify within the EV and renewable energy sector in the Sector Rotation Strategy.
b) Monitor policy changes and adjust strategies accordingly, especially for long-term positions.
c) Consider hedging with positions in traditional automotive or energy companies.
d) For the Sector Rotation Strategy, include a broader range of sectors (e.g., technology, consumer discretionary) to reduce EV-specific risk.

5. Execution Risk:
Monthly strategies may miss out on short-term opportunities or fail to react quickly to sudden market changes.

Mitigation Strategies:
a) Implement a hybrid approach that allows for some flexibility between monthly rebalancing periods.
b) Set up alerts for significant price movements or news events that may warrant immediate action.
c) Consider adding weekly reviews to complement the monthly trading frequency.
d) For all strategies, establish clear rules for when to deviate from the monthly schedule (e.g., stock price movement exceeding a certain threshold).

6. Options-Related Risks:
The Monthly Covered Call Strategy and Put Protection in Dollar-Cost Averaging strategy face risks related to options pricing and execution.

Mitigation Strategies:
a) Carefully select strike prices and expiration dates based on thorough technical and fundamental analysis.
b) Monitor implied volatility and adjust options strategies accordingly.
c) Consider rolling options before expiration to manage assignment risk.
d) For the Monthly Covered Call Strategy, use a dynamic strike price selection based on current volatility levels and technical resistance

# 주식거래봇 2

````
pip install tools sec-api unstructured
```

In [24]:
import warnings
warnings.filterwarnings('ignore')
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
import os
from langchain_anthropic import ChatAnthropic
from langchain.schema import SystemMessage, HumanMessage
from dotenv import load_dotenv
import os

In [25]:
import json
import os
import requests
from crewai import Agent, Task
from langchain.tools import tool
from unstructured.partition.html import partition_html
from langchain.tools import tool
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from sec_api import QueryApi

class DataTools():
    @tool("get historical stock price")
    def get_stock_price(ticker: str, start_date: str, end_date: str) -> str:
        """
        Get the stock price for a given ticker symbol and date range.

        Args:
            ticker (str): {stock_selection} (e.g., 'AAPL' for Apple Inc.)
            start_date (str): The start date in the format 'YYYY-MM-DD'
            end_date (str): The end date in the format 'YYYY-MM-DD'

        Returns:
            str: The stock price data for the given ticker and date range.
        """        
        import yfinance as yf
        import datetime

        today = datetime.datetime.now()
        one_year_ago = today - datetime.timedelta(days=365)

        stock_data = yf.download(ticker, start=one_year_ago, end=today)
        market_data = yf.download('^GSPC', start=one_year_ago, end=today)
        monthly_data = stock_data.resample('M').agg({
            'Close': 'last',  # 월말 종가
            'Volume': 'sum'   # 월간 누적 거래량
        })
        monthly_market = market_returns.resample('M').mean()  
        monthly_data['3M_Close'] = monthly_data['Close'].rolling(window=3).mean().shift(-2)        # 3개월 이동평균 종가 계
        risk_free_rate = 0.02 / 12  # 연 2% 무위험 수익률을 월별로 환산 # 샤프지수 계산
        monthly_data['Sharpe_Ratio'] = (monthly_data['Close'].pct_change() - risk_free_rate) / monthly_data['Close'].pct_change().std()
        monthly_data['Beta'] = stock_data['Close'].pct_change().resample('M').mean()/market_data['Close'].pct_change().resample('M').mean()
        return monthly_data

    @tool("get financial statement")        
    def get_financial_statement(ticker: str) -> str:
        '''
        Get the quarterly financial statement for a given ticker symbol.

        Args:
            ticker (str): {stock_selection} (e.g., 'AAPL' for Apple Inc.)

        Returns:
            str: quarterly_financials
        '''
        import pandas as pd
        import yfinance as yf
        ticker = yf.Ticker(ticker)
        return ticker.quarterly_financials.iloc[:,[0,4]]

class CalculatorTools():
    @tool("Make a calculation")
    def calculate(operation):
        """Useful to perform any mathematical calculations, 
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical 
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        return eval(operation)


class SearchTools():
    @tool("Search the internet")
    def search_internet(query):
        """Useful to search the internet 
        about a a given topic and return relevant results"""
        top_result_to_return = 4
        url = "https://google.serper.dev/search"
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        results = response.json()['organic']
        string = []
        for result in results[:top_result_to_return]:
            try:
                string.append('\n'.join([
                f"Title: {result['title']}", f"Link: {result['link']}",
                f"Snippet: {result['snippet']}", "\n-----------------"
            ]))
            except KeyError:
                next

        return '\n'.join(string)
    
    @tool("Search news on the internet")
    def search_news(query):
        """Useful to search news about a company, stock or any other
        topic and return relevant results"""""
        top_result_to_return = 4
        url = "https://google.serper.dev/news"
        payload = json.dumps({"q": query})
        headers = {
            'X-API-KEY': os.environ['SERPER_API_KEY'],
            'content-type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)
        results = response.json()['news']
        string = []
        for result in results[:top_result_to_return]:
            try:
                string.append('\n'.join([
                    f"Title: {result['title']}", f"Link: {result['link']}",
                    f"Snippet: {result['snippet']}", "\n-----------------"
                ]))
            except KeyError:
                next

        return '\n'.join(string)



In [26]:
class StockAnalysisTasks():
    def research(self, agent, company):
        return Task(
        description=dedent(f"""
        수집하고 요약하세요. 최근 뉴스 기사, 보도 자료 및 주식과
        해당 산업과 관련된 시장 분석 자료.
        중요한 이벤트, 시장 분위기 및 애널리스트 의견에
        특별한 주의를 기울이세요. 또한 실적 발표 등의 예정된
        이벤트도 포함하세요.

        최종 답변에는 반드시 최신 뉴스에 대한 포괄적인 요약,
        주목할 만한 시장 분위기 변화, 주가에 미칠 수 있는 잠재적 영향이
        포함되어야 합니다.
        또한 주식 티커도 반환해야 합니다.

        {self.__tip_section()}

        가능한 한 최신 데이터를 사용하세요.

        고객이 선택한 회사: {company}
        """),
        expected_output= "주식과 해당 산업과 관련된 최근 뉴스, 시장 분석, 이벤트를 요약한 포괄적인 보고서. 주목할 만한 시장 분위기 변화, \
        주가에 미칠 수 있는 잠재적 영향 및 주식 티커가 포함되어야 합니다.",
        agent=agent
    )
    
    def financial_analysis(self, agent):
        expected_output = "주요 재무 지표, 업계 동종 기업 및 시장 동향과의 비교, 강점, 약점, 전반적인 재무 건전성 등을 포함하여 \
        주식의 재무 건전성과 시장 성과를 분석한 포괄적인 보고서입니다."
        return Task(
            description=dedent(f"""
            주식의 재무 건전성과 시장 성과를 철저히 분석하세요.
            이에는 PER, EPS 성장률, 매출 추이, 부채비율 등
            주요 재무 지표 검토가 포함됩니다.
            또한 업계 동종 기업 및 전반적인 시장 동향과
            비교하여 주식 성과를 분석하세요.

            최종 보고서에는 제공된 요약을 확장하되,
            현재 시장 상황에서 주식의 재무 건전성, 강점과 약점,
            경쟁사 대비 위치 등에 대한 명확한 평가가 포함되어야 합니다.
            {self.__tip_section()}

            가능한 한 최신 데이터를 사용하세요.
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def filings_analysis(self, agent):
        expected_output = "경영진의 분석, 재무제표, 내부자 거래 활동, 공시 위험 등 최신 10-Q 및 10-K 보고서의 주요 부분에서 \
        중요한 발견 사항을 강조한 확장 보고서입니다. 고객을 위한 위험 요인 또는 긍정적 지표도 포함되어 있습니다."
        return Task(
            description=dedent(f"""
            EDGAR에서 해당 주식의 최신 10-Q 및 10-K 보고서를
            분석하세요.
            경영진의 분석, 재무제표, 내부자 거래 활동,
            공시된 위험 등 주요 부분에 중점을 두세요.
            주식 미래 성과에 영향을 줄 수 있는 관련 데이터와
            통찰력을 추출하세요.

            최종 답변에는 이러한 보고서에서 중요한 발견 사항을
            강조한 확장 보고서가 포함되어야 합니다.
            고객을 위한 위험 요인 또는 긍정적 지표도 포함하세요.
            {self.__tip_section()}
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def recommend(self, agent):
        expected_output = "재무 애널리스트와 리서치 애널리스트의 분석을 종합하여 재무 건전성, 시장 분위기, \
        EDGAR 보고서의 정성적 데이터 등 모든 측면을 고려한 포괄적인 투자 권장 보고서입니다. \
        보고서에는 명확한 투자 입장과 전략, 지원 근거, 내부자 거래 활동 섹션, 예정된 실적 발표 등이 포함되어야 하며 고객을 위해 잘 구성되어야 합니다."
        return Task(
            description=dedent(f"""
            재무 애널리스트와 리서치 애널리스트가 제공한 분석을
            검토하고 종합하세요.
            이러한 통찰력을 결합하여 포괄적인 투자 권장 사항을
            작성하세요.

            재무 건전성, 시장 분위기, EDGAR 보고서의 정성적
            데이터 등 모든 측면을 고려해야 합니다.

            내부자 거래 활동 섹션과 예정된 실적 발표 등도
            포함해야 합니다.

            최종 답변에는 고객을 위한 권장 사항이 포함되어야 합니다.
            지원 근거와 함께 명확한 투자 입장과 전략을 제시하는
            매우 상세한 전체 보고서여야 합니다.
            고객을 위해 예쁘고 잘 구성된 형식이어야 합니다.
            {self.__tip_section()}
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def __tip_section(self):
        return "최선을 다해 작업하면 10,000달러의 수수료를 드리겠습니다!"

In [27]:
from crewai import Task
from textwrap import dedent


class StockAnalysisTasks():
    def research(self, agent, company):
        return Task(
        description=dedent(f"""
        주식 및 해당 산업과 관련된 최근 뉴스 기사, 보도자료, 시장 분석 자료를 수집하고 요약합니다.
        중요한 이벤트, 시장 분위기, 애널리스트 의견에 특별한 주의를 기울입니다.
        실적 발표 등 예정된 이벤트도 포함합니다.
        최종 답변은 다음을 포함하는 보고서여야 합니다:
            최신 뉴스에 대한 포괄적인 요약,
            주목할 만한 시장 분위기 변화,
            주가에 미칠 수 있는 잠재적 영향
        주식 티커도 반환해야 합니다.

        {self.__tip_section()}

        가능한 한 최신 데이터를 사용해야 합니다.
        고객이 선택한 company를 지정해야 합니다.: {company}
        """),
        expected_output= "A comprehensive report summarizing recent news, market analyses,\
        and events related to the stock and its industry, including notable shifts in market sentiment, \
        potential impacts on the stock, and the stock ticker.",
        agent=agent
    )
    
    def financial_analysis(self, agent):
        expected_output = "A comprehensive report analyzing the stock's financial health and market performance, \
        including key financial metrics, comparison with industry peers and \
        market trends, strengths, weaknesses, and overall financial standing."
        return Task(
            description=dedent(f"""
            주식의 재무 건전성과 시장 성과를 철저히 분석합니다.
            PER, EPS 성장률, 매출 추이, 부채비율 등 주요 재무지표를 검토합니다.
            업계 동종기업 및 전반적인 시장 동향과 비교하여 주식 성과를 분석합니다.
            최종 보고서에는 제공된 요약을 확장하되, 다음 사항을 명확히 평가해야 합니다:
                주식의 재무 상황,
                강점과 약점,
                현재 시장에서 경쟁사 대비 위치.{self.__tip_section()}
            가능한 한 최신 데이터를 사용해야 합니다.
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def filings_analysis(self, agent):
        expected_output = "An expanded report highlighting significant findings from the latest 10-Q and 10-K filings, \
        including key insights from sections like Management's Discussion and Analysis, financial statements, \
        insider trading activity, and disclosed risks, as well as any red flags or positive indicators for the customer."
        return Task(
            description=dedent(f"""
            최근 주가의 흐름을 분석합니다.
            경영진 분석, 재무제표, 내부자 거래 활동, 공시된 위험 등 핵심 섹션에 중점을 둡니다.
            주식 미래 성과에 영향을 줄 수 있는 관련 데이터와 통찰력을 추출합니다.
            최종 답변은 이러한 보고서에서 중요한 발견 사항을 강조한 확장 보고서여야 합니다.
            고객을 위한 위험 요인(레드 플래그) 또는 긍정적 지표를 포함해야 합니다.{self.__tip_section()}
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def recommend(self, agent):
        expected_output = "A comprehensive investment recommendation report \
        that synthesizes the analyses from the Financial Analyst and Research Analyst, \
        considering all aspects such as financial health, market sentiment, and qualitative data from EDGAR filings. \
        The report should include a clear investment stance and strategy with supporting evidence, a section on insider trading activity, \
        upcoming events like earnings, and be well-formatted for the customer."
        return Task(
            description=dedent(f"""
            재무 애널리스트와 리서치 애널리스트가 제공한 분석을 검토하고 종합합니다.
            이러한 통찰력을 결합하여 포괄적인 투자 권장 사항을 작성합니다.
            재무 건전성, 시장 분위기, 실적 보고서의 정성적 데이터 등 모든 측면을 고려해야 합니다.
            내부자 거래 활동 섹션과 예정된 실적 발표 등을 포함해야 합니다.
            최종 답변은 고객을 위한 권장 사항이어야 합니다.
            매우 상세한 전체 보고서 형식이어야 하며, 지원 근거와 함께 명확한 투자 입장과 전략을 제시해야 합니다.
            고객을 위해 예쁘고 잘 구성된 형식이어야 합니다.
            {self.__tip_section()}
            """),
            expected_output=expected_output,
            agent=agent
        )
    
    def __tip_section(self):
        return "If you do your BEST WORK, I'll give you a $10,000 commission!"

In [28]:
from crewai import Agent
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os
from langchain_community.vectorstores.redis import Redis
from langchain.embeddings import OpenAIEmbeddings

load_dotenv()
chat = ChatGroq(model_name="mixtral-8x7b-32768", groq_api_key=os.getenv("GROQ_API_KEY"), temperature=0)
clud = ChatAnthropic(model="claude-3-sonnet-20240229", anthropic_api_key=os.getenv("Anthropic_API_KEY"), temperature=0)

# Redis 벡터 스토어 초기화
embeddings = OpenAIEmbeddings()
vector_store = Redis(embedding= embeddings, redis_url="redis://localhost:6379", index_name="users")

class StockAnalysisAgents():
    def local_expert(self):
        return Agent(
            role='The Best Financial Analyst',
            goal="""Impress all customers with your financial data and market trends analysis""",
            backstory="""The most seasoned financial analyst with lots of expertise in stock market analysis and investment strategies that is working for a super important customer.""",
            verbose=True,
            llm=chat.create_chain(vector_store),  # <----- Using ChatGroq with vector store
            tools=[
                SearchTools.search_internet,
                CalculatorTools.calculate,
                DataTools.get_stock_price,
                DataTools.get_financial_statement
            ]
        )
    
    def research_analyst(self):
        return Agent(
            role='Staff Research Analyst',
            goal="""Being the best at gather, interpret data and amaze your customer with it""",
            backstory="""Known as the BEST research analyst, you're skilled in sifting through news, company announcements, and market sentiments. Now you're working on a super important customer""",
            verbose=True,
            llm=clud.create_chain(vector_store),  # <----- Using AnthropicChatModel with vector store
            tools=[
                SearchTools.search_internet,
                SearchTools.search_news,
                YahooFinanceNewsTool(),
                DataTools.get_stock_price,
                DataTools.get_financial_statement
            ]
        )


    def investment_advisor(self):
        return Agent(
            role='Private Investment Advisor',
            goal="""Impress your customers with full analyses over stocks and completer investment recommendations""",
            backstory="""You're the most experienced investment advisor and you combine various analytical insights to formulate strategic investment advice. You are now working for a super important customer you need to impress.""",
            verbose=True,
            llm=clud.create_chain(vector_store),  # <----- Using ChatAnthropic with vector store
            tools=[
                SearchTools.search_internet,
                SearchTools.search_news,
                CalculatorTools.calculate,
                YahooFinanceNewsTool()
            ]
        )

class FinancialCrew:
    def __init__(self, company):
        self.company = company

    def run(self):
        agents = StockAnalysisAgents()
        tasks = StockAnalysisTasks()

        research_analyst_agent = agents.research_analyst()
        financial_analyst_agent = agents.local_expert()
        investment_advisor_agent = agents.investment_advisor()

        research_task = tasks.research(research_analyst_agent, self.company)
        financial_task = tasks.financial_analysis(financial_analyst_agent)
        filings_task = tasks.filings_analysis(financial_analyst_agent)
        recommend_task = tasks.recommend(investment_advisor_agent)

        crew = Crew(
            agents=[
                research_analyst_agent,
                financial_analyst_agent,
                investment_advisor_agent
            ],
            tasks=[
                research_task,
                financial_task,
                filings_task,
                recommend_task
            ],
            manager_llm=chat,  # <----- Using ChatGroq as the manager
            verbose=True
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to Financial Analysis Crew")
    print('-------------------------------')
    company = input(
        dedent("""
      What is the company you want to analyze?
    """))
    financial_crew = FinancialCrew(company)
    result = financial_crew.run()
    print("\n\n########################")
    print("## Here is the Report")
    print("########################\n")
    print(result)

ConnectionError: Error 10061 connecting to localhost:6379. 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다.

In [ ]:
from crewai import Agent
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os
load_dotenv()

load_dotenv()
chat = ChatGroq(model_name="mixtral-8x7b-32768", groq_api_key=os.getenv("GROQ_API_KEY"), temperature=0)
clud = ChatAnthropic(model="claude-3-sonnet-20240229",anthropic_api_key=os.getenv("Anthropic_API_KEY"), temperature=0)


class StockAnalysisAgents():
    def local_expert(self):
        return Agent(
            role='The Best Financial Analyst',
            goal="""Impress all customers with your financial data and market trends analysis""",
            backstory="""The most seasoned financial analyst with lots of expertise in stock market analysis and investment strategies that is working for a super important customer.""",
            verbose=True,
            llm=chat,  # <----- Using ChatGroq
            tools=[
                SearchTools.search_internet,
                CalculatorTools.calculate,
                DataTools.get_stock_price,
                DataTools.get_financial_statement
            ]
        )

    def research_analyst(self):
        return Agent(
            role='Staff Research Analyst',
            goal="""Being the best at gather, interpret data and amaze your customer with it""",
            backstory="""Known as the BEST research analyst, you're skilled in sifting through news, company announcements, and market sentiments. Now you're working on a super important customer""",
            verbose=True,
            llm=chat,  # <----- Using ChatGroq
            tools=[
                SearchTools.search_internet,
                SearchTools.search_news,
                YahooFinanceNewsTool(),
                DataTools.get_stock_price,
                DataTools.get_financial_statement
            ]
        )

    def investment_advisor(self):
        return Agent(
            role='Private Investment Advisor',
            goal="""Impress your customers with full analyses over stocks and completer investment recommendations""",
            backstory="""You're the most experienced investment advisor and you combine various analytical insights to formulate strategic investment advice. You are now working for a super important customer you need to impress.""",
            verbose=True,
            llm=chat,  # <----- Using ChatGroq
            tools=[
                SearchTools.search_internet,
                SearchTools.search_news,
                CalculatorTools.calculate,
                YahooFinanceNewsTool()
            ]
        )

class FinancialCrew:
    def __init__(self, company):
        self.company = company

    def run(self):
        agents = StockAnalysisAgents()
        tasks = StockAnalysisTasks()

        research_analyst_agent = agents.research_analyst()
        financial_analyst_agent = agents.local_expert()
        investment_advisor_agent = agents.investment_advisor()

        research_task = tasks.research(research_analyst_agent, self.company)
        financial_task = tasks.financial_analysis(financial_analyst_agent)
        filings_task = tasks.filings_analysis(financial_analyst_agent)
        recommend_task = tasks.recommend(investment_advisor_agent)

        crew = Crew(
            agents=[
                research_analyst_agent,
                financial_analyst_agent,
                investment_advisor_agent
            ],
            tasks=[
                research_task,
                financial_task,
                filings_task,
                recommend_task
            ],
            manager_llm=chat,  # <----- Using ChatGroq as the manager
            verbose=True
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to Financial Analysis Crew")
    print('-------------------------------')
    company = input(
        dedent("""
      What is the company you want to analyze?
    """))
    financial_crew = FinancialCrew(company)
    result = financial_crew.run()
    print("\n\n########################")
    print("## Here is the Report")
    print("########################\n")
    print(result)

In [ ]:
# from crewai import Agent
# from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
# from langchain.chat_models import ChatOpenAI
# from dotenv import load_dotenv
# from langchain_groq import ChatGroq
# import os
# load_dotenv()
# # gpt = ChatOpenAI(model='gpt-3.5', api_key=os.getenv("OPENAI_API_KEY")) # Loading GPT-3.5
# # clud = ChatAnthropic(model="claude-3-sonnet-20240229",anthropic_api_key=os.getenv("Anthropic_API_KEY"), temperature=0)
# chat = ChatGroq(model_name="mixtral-8x7b-32768",groq_api_key="GROQ_API_KEY",temperature=0)
# class StockAnalysisAgents():

#     def local_expert(self):
#         return Agent(
#                   role='The Best Financial Analyst',
#                   goal="""Impress all customers with your financial data 
#                   and market trends analysis""",
#                   backstory="""The most seasoned financial analyst with 
#                   lots of expertise in stock market analysis and investment
#                   strategies that is working for a super important customer.""",
#                   verbose=True,
#                   llm=chat, # <----- passing our llm reference here
#                   tools=[
#                     SearchTools.search_internet,
#                     CalculatorTools.calculate,
#                     # SECTools.search_10q,
#                     # SECTools.search_10k
#                     DataTools.get_stock_price,
#                     DataTools.get_financial_statement
#                   ]
#                 )
#     def research_analyst(self):
#         return Agent(
#                   role='Staff Research Analyst',
#                   goal="""Being the best at gather, interpret data and amaze
#                   your customer with it""",
#                   backstory="""Known as the BEST research analyst, you're
#                   skilled in sifting through news, company announcements, 
#                   and market sentiments. Now you're working on a super 
#                   important customer""",
#                   verbose=True,
#                   LLM=chat,
#                   tools=[
#                     SearchTools.search_internet,
#                     SearchTools.search_news,
#                     YahooFinanceNewsTool(),
#                     DataTools.get_stock_price,
#                     DataTools.get_financial_statement
#                   ]
#               )
#     def investment_advisor(self):
#         return Agent(
#                   role='Private Investment Advisor',
#                   goal="""Impress your customers with full analyses over stocks
#                   and completer investment recommendations""",
#                   backstory="""You're the most experienced investment advisor
#                   and you combine various analytical insights to formulate
#                   strategic investment advice. You are now working for
#                   a super important customer you need to impress.""",
#                   verbose=True,
#                   LLM=chat,
#                   tools=[
#                     SearchTools.search_internet,
#                     SearchTools.search_news,
#                     CalculatorTools.calculate,
#                     YahooFinanceNewsTool()
#                   ]
#                 )
    
# from crewai import Crew
# from textwrap import dedent
# from dotenv import load_dotenv
# load_dotenv()

# # from stock_analysis_agents import StockAnalysisAgents
# # from stock_analysis_tasks import StockAnalysisTasks


# class FinancialCrew:
#     def __init__(self, company):
#         self.company = company
#     def run(self):
#         agents = StockAnalysisAgents()
#         tasks = StockAnalysisTasks()

#         research_analyst_agent = agents.research_analyst()
#         financial_analyst_agent = agents.local_expert()
#         investment_advisor_agent = agents.investment_advisor()

#         research_task = tasks.research(research_analyst_agent, self.company)
#         financial_task = tasks.financial_analysis(financial_analyst_agent)
#         filings_task = tasks.filings_analysis(financial_analyst_agent)
#         recommend_task = tasks.recommend(investment_advisor_agent)

#         crew = Crew(
#           agents=[
#             research_analyst_agent,
#             financial_analyst_agent,
#             investment_advisor_agent
#           ],
#           tasks=[
#             research_task,
#             financial_task,
#             filings_task,
#             recommend_task
#           ],
#           manager_llm=clud,
#           verbose=True
#         )

#         result = crew.kickoff()
#         return result
    
# if __name__ == "__main__":
#     print("## Welcome to Financial Analysis Crew")
#     print('-------------------------------')
#     company = input(
#         dedent("""
#       What is the company you want to analyze?
#     """))
#     financial_crew = FinancialCrew(company)
#     result = financial_crew.run()
#     print("\n\n########################")
#     print("## Here is the Report")
#     print("########################\n")
#     print(result)

In [ ]:
# class SECTools():
#     def __download_form_html(url):
#         headers = {
#       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#       'Accept-Encoding': 'gzip, deflate, br',
#       'Accept-Language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
#       'Cache-Control': 'max-age=0',
#       'Dnt': '1',
#       'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120"',
#       'Sec-Ch-Ua-Mobile': '?0',
#       'Sec-Ch-Ua-Platform': '"macOS"',
#       'Sec-Fetch-Dest': 'document',
#       'Sec-Fetch-Mode': 'navigate',
#       'Sec-Fetch-Site': 'none',
#       'Sec-Fetch-User': '?1',
#       'Upgrade-Insecure-Requests': '1',
#       'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
#     }
#         response = requests.get(url, headers=headers)
#         return response.text
    
#     def __embedding_search(url, ask):
#         text = SECTools.__download_form_html(url)
#         elements = partition_html(text=text)
#         content = "\n".join([str(el) for el in elements])
#         text_splitter = CharacterTextSplitter(
#             separator = "\n",
#             chunk_size = 1000,
#             chunk_overlap  = 150,
#             length_function = len,
#             is_separator_regex = False,
#         )
#         docs = text_splitter.create_documents([content])
#         retriever = FAISS.from_documents(
#           docs, OpenAIEmbeddings()
#         ).as_retriever()
#         answers = retriever.get_relevant_documents(ask, top_k=4)
#         answers = "\n\n".join([a.page_content for a in answers])
#         return answers
    
#     @tool("Search 10-Q")
#     def search_10q(query):
#         """
#         Useful to search information from the latest 10-Q form for a given stock.
#         The input to this tool should be a pipe (|) separated text of length two, 
#         representing the stock ticker you are interested and what question you have from it.
#         For example, 'AAPL|what was last quarter's revenue'.
#         """
#         stock, ask = query.split("|")
#         queryApi = QueryApi(api_key=os.environ['SEC_API_KEY'])
#         query = {
#           "query": {
#             "query_string": {
#               "query": f"ticker:{stock} AND formType:\"10-Q\""
#             }
#           },
#           "from": "0",
#           "size": "1",
#           "sort": [{ "filedAt": { "order": "desc" }}]
#         }

#         fillings = queryApi.get_filings(query)['filings']
#         if len(fillings) == 0:
#             return "Sorry, I couldn't find any filling for this stock, check if the ticker is correct."
#         link = fillings[0]['linkToFilingDetails']
#         answer = SECTools.__embedding_search(link, ask)
#         return answer

#     @tool("Search 10-K")
#     def search_10k(query):
#         """
#         Useful to search information from the latest 10-K form for a given stock.
#         The input to this tool should be a pipe (|) separated text of length two, 
#         representing the stock ticker you are interested, what question you have from it.
#         For example, 'AAPL|what was last year's revenue'.
#         """
#         stock, ask = query.split("|")
#         queryApi = QueryApi(api_key=os.environ['SEC_API_KEY'])
#         query = {
#           "query": {
#             "query_string": {
#               "query": f"ticker:{stock} AND formType:\"10-K\""
#             }
#           },
#           "from": "0",
#           "size": "1",
#           "sort": [{ "filedAt": { "order": "desc" }}]
#         }

#         fillings = queryApi.get_filings(query)['filings']
#         if len(fillings) == 0:
#             return "Sorry, I couldn't find any filling for this stock, check if the ticker is correct."
#         link = fillings[0]['linkToFilingDetails']
#         answer = SECTools.__embedding_search(link, ask)
#         return answer   
